In [11]:
import os
import math
import torch
from dataclasses import dataclass, field

from code.train import train
from code.optimizers import Optimizer
# from code.problems import Problem
from code.problem import Loss
from code.datasets import Dataset
from code.models import Model

# %matplotlib widget
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import itertools
def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [13]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [14]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

# Mean Estimation

In [15]:
@dataclass
class BaseConfig():
    n_iters: int = 50
    n_peers: int = 15
    seed: int = 0

    model: Model = field(default_factory=lambda: Model.Mean)
    loss: Loss = field(default_factory=lambda: Loss.MSE)
    dataset: Dataset = field(default_factory=lambda: Dataset.Normal)

    n_samples: int = 1000
    mu_normal: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 100
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [16]:
parameters = {
    'mu_normal': [0.001],
    # 'mu_normal': [0.001, 0.01, 0.1],
    # 'seed': [0],
    }

In [ ]:
for cfg in product_dict(**parameters):

    config = BaseConfig(**cfg)    
    config.optimizer = Optimizer.SGD
    config.true_weights = True
    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = True
#     config.md_n_iters_ = 20
#     config.md_lr_ = 0.05
#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
#     %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.SGD
#     config.true_weights = False
#     os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
#     %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = False
#     config.md_n_iters_ = 20
#     config.md_lr_ = 0.5
#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
#     %time train(config)

# 4 classes MNIST

In [6]:
@dataclass
class BaseConfig():
    n_iters: int = 5
    n_peers: int = 20
    seed: int = 0

    model: Model = field(default_factory=lambda: Model.Net)
    loss: Loss = field(default_factory=lambda: Loss.CrossEntropy)
    dataset: Dataset = field(default_factory=lambda: Dataset.MNIST)

    n_samples: int = 200
    h_ratio: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 20
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [7]:
parameters = {
    'h_ratio': [0.99],
    # 'h_ratio': [0.99, 0.9, 0.7],
    # 'seed': [0],
    }

In [10]:
for cfg in product_dict(**parameters):

    config = BaseConfig(**cfg)
    config.optimizer = Optimizer.SGD
    config.true_weights = True
    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = True
#     config.md_n_iters_ = 20
#     config.md_lr_ = 0.05
#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
#     %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.SGD
#     config.true_weights = False
#     os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
#     %time train(config)


#     config = Config(mu_normal=mu)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = False
#     config.md_n_iters_ = 20
#     config.md_lr_ = 0.5
#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
#     %time train(config)

calc3 18
actual3 18
calc1 200
actual1 200
test len  200
rank=0 destroy complete    
rank=1 destroy complete
rank=2 destroy complete
rank=3 destroy complete
rank=4 destroy complete
rank=5 destroy complete
rank=6 destroy complete
rank=7 destroy complete
rank=8 destroy complete
rank=9 destroy complete
rank=10 destroy complete
rank=11 destroy complete
rank=12 destroy complete
rank=13 destroy complete
rank=14 destroy complete
rank=15 destroy complete
rank=16 destroy complete
rank=17 destroy complete
rank=18 destroy complete
rank=19 destroy complete
CPU times: user 24.5 ms, sys: 26.1 ms, total: 50.6 ms
Wall time: 36.8 s
